In [ ]:
import os
import pandas as pd

In [ ]:
filepath = '/media/sf_VBox_Shared/Arabic/Fiqh/2018-04-24-Fiqh/'

In [ ]:
filename = os.path.join(filepath, 'Fiqh', 'Fiqh1.txt')

In [ ]:
metadata_dict = {}

for filename in os.listdir( os.path.join(filepath, 'Fiqh')):
    with open(os.path.join(filepath, 'Fiqh', filename)) as f:
        metadata = {}
        for line in f.readlines():
            # TODO: metadata is sometimes inconsistent, (missing # before META,
            # and fields not separated by :: but single :)
            if line.startswith('#META#'):
                splitted = line.split(u'::')
                if(len(splitted)==2):
                    name, value = line.split(u'::')

                    value = value.strip()
                    name = name.strip()

                    # only save metadata that has a value
                    #if value != 'NODATA':
                    _, name = name.split(u' ', 1)
                    name = name.replace(u' ', u'_')
                    # remove left to right mark
                    name = name.replace(u"\u200F", u'')
                    name = name.split(u'.')[-1]
                    metadata[name] = value
        metadata_dict[filename] = metadata

In [ ]:
metadata_df = pd.DataFrame.from_dict(metadata_dict, orient='index')

In [ ]:
metadata_df.index.name = 'filename'

In [ ]:
metadata_df.to_csv(os.path.join(filepath, 'metadata-from-files.csv'))

In [ ]:
metadata_df.info()

In [ ]:
sorted(metadata_df.columns)